In [1]:
import pandas as pd
import os

from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error
from xgboost import XGBRegressor

# Read the data
dir = os.getcwd()
X = pd.read_csv(os.path.join(dir,'input', 'train.csv'), index_col='Id')
X_test_full = pd.read_csv(os.path.join(dir,'input', 'test.csv'), index_col='Id')

# Remove rows with missing target, separate target from predictors
X.dropna(axis=0, subset=['SalePrice'], inplace=True)
y = X.SalePrice              
X.drop(['SalePrice'], axis=1, inplace=True)

# Break off validation set from training data
X_train_full, X_valid_full, y_train, y_valid = train_test_split(X, y, train_size=0.8, test_size=0.2,
                                                                random_state=0)

# Select categorical columns with relatively low cardinality (convenient but arbitrary)
low_cardinality_cols = [cname for cname in X_train_full.columns if X_train_full[cname].nunique() < 10 and 
                        X_train_full[cname].dtype == "object"]

# Select numeric columns
numeric_cols = [cname for cname in X_train_full.columns if X_train_full[cname].dtype in ['int64', 'float64']]

# Keep selected columns only
my_cols = low_cardinality_cols + numeric_cols
X_train = X_train_full[my_cols].copy()
X_valid = X_valid_full[my_cols].copy()
X_test = X_test_full[my_cols].copy()

# One-hot encode the data (to shorten the code, we use pandas)
X_train = pd.get_dummies(X_train)
X_valid = pd.get_dummies(X_valid)
X_test = pd.get_dummies(X_test)
X_train, X_valid = X_train.align(X_valid, join='left', axis=1)
X_train, X_test = X_train.align(X_test, join='left', axis=1)



# Define the model
fp_model = XGBRegressor(n_estimators=1000, learning_rate=0.05, early_stopping_rounds=5)

# Fit the model
fp_model.fit(X_train, y_train,
               eval_set=[(X_valid, y_valid)],
               verbose=False)

# Get predictions
predictions_y = fp_model.predict(X_valid)

# Calculate MAE
mae = mean_absolute_error(predictions_y, y_valid)

# Uncomment to print MAE
print("Mean Absolute Error:" , mae)

# Predict values
preds_test = fp_model.predict(X_test)

# Save test predictions to file
output = pd.DataFrame({'Id': X_test.index,
                       'SalePrice': preds_test})
output.to_csv('submission.csv', index=False)

Mean Absolute Error: 17032.76953125
